Testing data positive feedback dynamic (20%)


In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')
image_2048 = np.load('../Data/npy/Image_2048D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [4]:
print('user_following shape ',user_following.shape)
print('image_2048 shape ',image_2048.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (1489, 88)
image_2048 shape  (88, 2048)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)


In [5]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (1489, 17)


In [6]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)

In [7]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  34


In [8]:
over_10 = 0
over_8 = 0
over_12 = 0
user_idx_over10 = []
for i in range(len(num_of_follower)):
    num = num_of_follower[i]
    if num >= 10:
        over_10 += 1
        user_idx_over10.append(i)
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)

over 8: 1073
over 10: 792
over 12: 499


In [9]:
len(user_idx_over10)

792

# Training data and Testing data

In [10]:
test_amount = 150
yt_test_amount = 20

In [11]:
#user_idx = [i for i in range(len(user_following))]
user_idx = user_idx_over10
#test_idx is the number of user for testing
test_idx = random.sample(user_idx,test_amount)

In [12]:
# Training  and Testing
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        # random choose 2 true and 8 false for test 
        if len(temp_t)*0.2 < 1:
            t_for_test = random.sample(temp_t,1)
            f_for_test  = random.sample(temp_f,yt_test_amount-1)
        else:
            if len(temp_t)*0.2 > 5:
                t_num = int(round(float(len(temp_t)*0.2)))
            else:      
                t_num = 5
            #t_num = int(round(float(len(temp_t)*0.2)))
            t_for_test = random.sample(temp_t,t_num)
            f_for_test  = random.sample(temp_f,yt_test_amount-t_num)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [13]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/150
avg

5.0

In [14]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1489
The length of train_f: 1489
The length of test_t: 150
The length of test_f: 150


# Recommendation  Module

In [15]:
"""
n: the number of users
m: the number of YouTubers
k: latent dims
l: feature dims
"""
n = 1489
m = 88  
k = 128
l = 2048 

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None,l))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))


image_i = tf.placeholder(tf.float32, shape=(1,l))
image_j = tf.placeholder(tf.float32, shape=(1,l))

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [n, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    W1 = tf.get_variable("W1", [n, k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [k,k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [m,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [k,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
    
    

    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    Beta = tf.get_variable("beta", [n,l],
                             # initializer=tf.contrib.layers.xavier_initializer())
                                     initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i) 
vj = tf.nn.embedding_lookup(item_latent, j)

w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
wu = Wu
#wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
wa = Wa
#wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(k*k)
wv = Wv
#wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [16]:
a_list=tf.Variable([])
q = tf.constant(0)
def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])
def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    
    a_list = tf.concat([a_list,[(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
        tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q]]],0)
    q += 1
    return q,  a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

a_list_soft=tf.nn.softmax(a_list)


aux_np = tf.expand_dims(tf.zeros(128),0) #dimension (1,32)
q = tf.constant(0)
def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
    q += 1
    return q, aux_np

_,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

"""
for q in range(3): #取q個auxliary item
    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
"""


aux_np+=u #user_latent factor + sum (alpha*auxilary)
aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np

#矩陣中對應函數各自相乘
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,image_i, transpose_b=True)
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,image_j, transpose_b=True)

xuij = xui- xuj

l2_norm = tf.add_n([
            0.001 * tf.reduce_sum(tf.multiply(u, u)),
            0.001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.001 * tf.reduce_sum(tf.multiply(w1, w1)),
            0.001 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.001 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.001 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.001 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.1 * tf.reduce_sum(tf.multiply(beta,beta)),
            
          ])

loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

Instructions for updating:
Use `tf.cast` instead.


In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0=time.time()

#use_true=init_list_of_objects(136)
#use_test=init_list_of_objects(136)

train_pair_t=[] #positive feedback
train_pair_f=[] #negative feedback
train_yes_id=[] 
for q in range(5):
    print('Iteraction:',q)
    train_auc=0
    total_loss=0
    xuij_auc=0
    length = 0
    for z in range(1489):
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes=[]
        yesr=[]
        
        
        sample=random.sample(train_t[z],len(train_t[z])) #隨機選3個sample true's YouTuber
        train_yes_id.append(sample) #sample全部丟進去
        
        #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
        #change
        r_3=np.zeros(len(sample)) 
        alpha_history = []
        a_list_history = []
        U_history = []
        Y_history = []
        
        #print(len(sample))
        #check if all YouTuber are in train_t or train_f
        #if len(train_t[z])+len(train_f[z]) != 88:
            #print(z,len(train_t[z])+len(train_f[z]))
         
        for k in range(len(sample)):
            yes.append(image_2048[sample[k]])
            yesr.append(YouTuber_category[sample[k]]*user_category_norm[z])
            #print('YouTuber_category ', YouTuber_category[sample[k]])
            #print('User_category ',user_category_norm[z])
        #print(len(yes))
        for k in range(len(sample)):
            r_3[k]=max(yesr[k])
        #print('r_3:',r_3)
        
        yes=np.array(yes)
        #print('user shape should be ',np.array([z]).shape)
        #print('xf shape should be ',yes.shape)
        #print('r shape should be ',np.array(r_3).shape)
        #print('l_id shape should be ',np.array(sample).shape)
        
        #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
        train_t_sample = random.sample(train_t[z],len(train_t[z]))
        #print('number of positive feedback', len(train_t[z]))
        for ta in train_t_sample:
            #ta=random.choice(train_t[z]) #ta is true positve photo
            train_pair_t.append(ta)
            image_1=np.expand_dims(image_2048[ta],0) #(1,2048)
            #print('Image_1 shape ',image_1.shape)
            train_f_sample = random.sample(train_f[z],20)
            for b in train_f_sample:
                #print('likes:',ta,';Not likes:',b)
                #b=random.choice(train_f[z])  #b is no feedback photo
                train_pair_f.append(b)
                image_2=np.expand_dims(image_2048[b],0) #(1,2048)
                #print('Image_2 shape',image_2.shape)
            
                #use_test[z].append(b)
                Uu,Yy,_a_list,r3,_auc, _loss,_=sess.run([user_latent,item_latent,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: yes , l_id:sample, l_id_len:[len(sample)],r:r_3,
                                        image_i:image_1,image_j:image_2})
                #print('User latent factor')
                #print(Uu.shape)
                #print(Uu)
                #print('Item latent factor')
                #print(Yy.shape)
                #print(Yy)
                U_history.append(Uu)
                Y_history.append(Yy)
                
                #print(XUIJ)
                #print('loss=',_loss)
                #print('auc=',_auc)
                #print(z,ta,b)
                #print('alpha list after softmax:',r3)
                #print('alpha list before softmax:',_a_list)
                a_list_history.append(_a_list)
                alpha_history.append(r3)
                train_auc+=_auc
                total_loss+=_loss
                length += 1
            #now1+=1
        
        """
        with open('../Data/alpha_txt/'+str(q)+'_'+str(z)+'.txt', 'w') as f:
            for idd in range(len(alpha_history)):
                f.write('softmax alpha:')
                f.write(str(alpha_history[idd])+'\n')
                f.write('before softmax:')
                f.write(str(a_list_history[idd])+'\n')
        """
        #np.savez('../Data/latent_factor/YRM_up10/'+str(q)+'_'+str(z)+'.npz', User=U_history, YouTuber=Y_history)
    
    #print('mine:',xuij_auc/136)    
    #print('a_list_soft:',r3)
    print("total_loss:-----------------", total_loss/length)
    print("train_auc:-------------------", train_auc/length)
    loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
    print('time:',time.time()-t0,' sec')
print('Total cost ',time.time()-t0,' sec')       

Iteraction: 0
total_loss:----------------- [[0.23415304]]
train_auc:------------------- 0.9368904109589041
time: 1559.3762130737305  sec
Iteraction: 1
total_loss:----------------- [[0.15129206]]
train_auc:------------------- 0.9757465753424658
time: 3146.8849227428436  sec
Iteraction: 2
total_loss:----------------- [[0.14163002]]
train_auc:------------------- 0.9812842465753424
time: 4760.222403287888  sec
Iteraction: 3
total_loss:----------------- [[0.13845695]]
train_auc:------------------- 0.9823869863013699
time: 6353.252666473389  sec
Iteraction: 4
total_loss:----------------- [[0.13763426]]
train_auc:------------------- 0.9828458904109589
time: 7912.437202215195  sec
Total cost  7912.437202215195  sec


In [18]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('time=',loss_acc_list[i][2])

Iteration: 0
loss= [[0.23415304]]
acc= 0.9368904109589041
time= 1559.3762130737305
Iteration: 1
loss= [[0.15129206]]
acc= 0.9757465753424658
time= 3146.8849227428436
Iteration: 2
loss= [[0.14163002]]
acc= 0.9812842465753424
time= 4760.222403287888
Iteration: 3
loss= [[0.13845695]]
acc= 0.9823869863013699
time= 6353.252666473389
Iteration: 4
loss= [[0.13763426]]
acc= 0.9828458904109589
time= 7912.437202215195


# Get latent factor and Each weight

In [19]:
U, Y, A, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, W1, Wu, Wy, Wa, Wv,Beta])

In [20]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)

User latent shape:  (1489, 128)
photo latent shape:  (88, 128)
Auxilary latent shape:  (88, 128)
W1 weight shape:  (1489, 128)
Wu weight shape: (128, 128)
Wy weight shape: (88, 128, 128)
Wa weight shape: (128, 128)
Wv weight shape: (128, 2048)
Beta shape: (1489, 2048)


# Testing Part

In [21]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au,np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa,
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av,np.expand_dims(image_2048[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    #print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], image_2048[k].T)
print(RS[s])

0 508
softmax alpha-------------- [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
1 1217
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
2 1236
softmax alpha-------------- [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
3 1315
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
4 1335
softmax alpha-------------- [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
5 306
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
6 502
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
7 823
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
8 754
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
9 209
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
10 928
softmax alpha-------------- [0.11111111 0.11111111 

softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
108 1067
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
109 1368
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
110 97
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
111 756
softmax alpha-------------- [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
112 431
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
113 1316
softmax alpha-------------- [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
114 602
softmax alpha-------------- [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
115 13
softmax alpha-------------- [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
116 1475
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
117 101
softmax alpha----------

In [22]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [42]:
target

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [23]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

num of positive data in testing: 750.0
total testing data: 3000


In [24]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [25]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/total
print('avg_accuarcy for count_0:',avg_acc)

[1, 17, 15, 14, 8]
[0, 4, 1, 16, 10]
[2, 19, 3, 4, 0]
[12, 11, 5, 9, 2]
[14, 4, 1, 6, 16]
[5, 6, 18, 1, 4]
[15, 9, 18, 8, 2]
[3, 6, 13, 17, 2]
[7, 18, 16, 11, 8]
[17, 7, 6, 2, 8]
[18, 1, 10, 19, 3]
[2, 6, 4, 8, 9]
[16, 5, 4, 12, 9]
[9, 5, 17, 14, 19]
[15, 17, 2, 7, 4]
[1, 15, 2, 7, 17]
[15, 0, 3, 5, 6]
[3, 2, 4, 1, 0]
[1, 2, 9, 16, 11]
[1, 4, 0, 2, 12]
[7, 0, 15, 14, 9]
[5, 0, 7, 14, 19]
[3, 4, 0, 1, 15]
[4, 11, 0, 18, 3]
[3, 8, 17, 6, 15]
[13, 1, 16, 6, 11]
[6, 17, 12, 2, 0]
[17, 8, 14, 5, 13]
[1, 9, 4, 0, 10]
[3, 4, 2, 18, 1]
[4, 3, 1, 2, 0]
[11, 16, 2, 12, 19]
[10, 3, 1, 17, 16]
[0, 16, 3, 15, 2]
[19, 1, 8, 7, 10]
[0, 8, 1, 3, 17]
[14, 0, 19, 2, 11]
[2, 10, 8, 5, 3]
[2, 10, 12, 15, 6]
[18, 4, 7, 10, 13]
[4, 2, 3, 1, 0]
[12, 15, 0, 1, 13]
[0, 1, 8, 3, 5]
[9, 11, 12, 19, 6]
[2, 0, 7, 12, 15]
[6, 17, 3, 9, 19]
[0, 2, 10, 3, 18]
[13, 8, 9, 5, 4]
[5, 14, 3, 1, 8]
[19, 11, 0, 15, 9]
[2, 12, 7, 1, 4]
[6, 9, 8, 14, 10]
[15, 6, 0, 11, 10]
[12, 13, 18, 19, 17]
[0, 11, 2, 9, 12]
[9, 3, 13, 11,

In [26]:
acc_0

338

In [27]:
total

3750

# Top 1 

In [28]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [29]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

[1]
[0]
[2]
[12]
[14]
[5]
[15]
[3]
[7]
[17]
[18]
[2]
[16]
[9]
[15]
[1]
[15]
[3]
[1]
[1]
[7]
[5]
[3]
[4]
[3]
[13]
[6]
[17]
[1]
[3]
[4]
[11]
[10]
[0]
[19]
[0]
[14]
[2]
[2]
[18]
[4]
[12]
[0]
[9]
[2]
[6]
[0]
[13]
[5]
[19]
[2]
[6]
[15]
[12]
[0]
[9]
[1]
[16]
[16]
[4]
[2]
[10]
[17]
[3]
[15]
[8]
[19]
[4]
[3]
[3]
[0]
[18]
[2]
[5]
[11]
[15]
[15]
[3]
[2]
[19]
[1]
[1]
[19]
[1]
[3]
[4]
[8]
[18]
[11]
[3]
[2]
[4]
[17]
[17]
[2]
[11]
[8]
[13]
[0]
[4]
[9]
[2]
[1]
[2]
[3]
[1]
[2]
[3]
[3]
[14]
[4]
[3]
[13]
[14]
[14]
[5]
[13]
[3]
[10]
[3]
[1]
[4]
[1]
[1]
[5]
[11]
[15]
[1]
[19]
[1]
[5]
[0]
[2]
[16]
[4]
[1]
[4]
[17]
[9]
[3]
[13]
[9]
[1]
[3]
[1]
[0]
[0]
[18]
[7]
[0]


In [30]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.5133333333333333 recall  0.10266666666666667


In [31]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

F1_score: 0.1711111111111111


# Top 3

In [32]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

[1, 17, 15]
[0, 4, 1]
[2, 19, 3]
[12, 11, 5]
[14, 4, 1]
[5, 6, 18]
[15, 9, 18]
[3, 6, 13]
[7, 18, 16]
[17, 7, 6]
[18, 1, 10]
[2, 6, 4]
[16, 5, 4]
[9, 5, 17]
[15, 17, 2]
[1, 15, 2]
[15, 0, 3]
[3, 2, 4]
[1, 2, 9]
[1, 4, 0]
[7, 0, 15]
[5, 0, 7]
[3, 4, 0]
[4, 11, 0]
[3, 8, 17]
[13, 1, 16]
[6, 17, 12]
[17, 8, 14]
[1, 9, 4]
[3, 4, 2]
[4, 3, 1]
[11, 16, 2]
[10, 3, 1]
[0, 16, 3]
[19, 1, 8]
[0, 8, 1]
[14, 0, 19]
[2, 10, 8]
[2, 10, 12]
[18, 4, 7]
[4, 2, 3]
[12, 15, 0]
[0, 1, 8]
[9, 11, 12]
[2, 0, 7]
[6, 17, 3]
[0, 2, 10]
[13, 8, 9]
[5, 14, 3]
[19, 11, 0]
[2, 12, 7]
[6, 9, 8]
[15, 6, 0]
[12, 13, 18]
[0, 11, 2]
[9, 3, 13]
[1, 4, 0]
[16, 1, 0]
[16, 2, 3]
[4, 1, 2]
[2, 12, 11]
[10, 19, 7]
[17, 5, 12]
[3, 18, 16]
[15, 1, 0]
[8, 1, 4]
[19, 2, 6]
[4, 3, 2]
[3, 17, 19]
[3, 4, 18]
[0, 8, 3]
[18, 5, 0]
[2, 9, 4]
[5, 7, 3]
[11, 2, 4]
[15, 2, 6]
[15, 5, 16]
[3, 14, 12]
[2, 4, 0]
[19, 14, 17]
[1, 0, 4]
[1, 4, 3]
[19, 9, 14]
[1, 4, 9]
[3, 2, 0]
[4, 3, 16]
[8, 2, 19]
[18, 3, 12]
[11, 5, 6]
[3, 1, 10]
[2, 1, 3]

In [33]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.5111111111111111 recall  0.30666666666666664


In [34]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

F1_score: 0.3833333333333333


In [35]:
F1_score(0.2222222222222222,0.0333333333333333)

0.05797101449275356

# Top 5

In [36]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

[1, 17, 15, 14, 8]
[0, 4, 1, 16, 10]
[2, 19, 3, 4, 0]
[12, 11, 5, 9, 2]
[14, 4, 1, 6, 16]
[5, 6, 18, 1, 4]
[15, 9, 18, 8, 2]
[3, 6, 13, 17, 2]
[7, 18, 16, 11, 8]
[17, 7, 6, 2, 8]
[18, 1, 10, 19, 3]
[2, 6, 4, 8, 9]
[16, 5, 4, 12, 9]
[9, 5, 17, 14, 19]
[15, 17, 2, 7, 4]
[1, 15, 2, 7, 17]
[15, 0, 3, 5, 6]
[3, 2, 4, 1, 0]
[1, 2, 9, 16, 11]
[1, 4, 0, 2, 12]
[7, 0, 15, 14, 9]
[5, 0, 7, 14, 19]
[3, 4, 0, 1, 15]
[4, 11, 0, 18, 3]
[3, 8, 17, 6, 15]
[13, 1, 16, 6, 11]
[6, 17, 12, 2, 0]
[17, 8, 14, 5, 13]
[1, 9, 4, 0, 10]
[3, 4, 2, 18, 1]
[4, 3, 1, 2, 0]
[11, 16, 2, 12, 19]
[10, 3, 1, 17, 16]
[0, 16, 3, 15, 2]
[19, 1, 8, 7, 10]
[0, 8, 1, 3, 17]
[14, 0, 19, 2, 11]
[2, 10, 8, 5, 3]
[2, 10, 12, 15, 6]
[18, 4, 7, 10, 13]
[4, 2, 3, 1, 0]
[12, 15, 0, 1, 13]
[0, 1, 8, 3, 5]
[9, 11, 12, 19, 6]
[2, 0, 7, 12, 15]
[6, 17, 3, 9, 19]
[0, 2, 10, 3, 18]
[13, 8, 9, 5, 4]
[5, 14, 3, 1, 8]
[19, 11, 0, 15, 9]
[2, 12, 7, 1, 4]
[6, 9, 8, 14, 10]
[15, 6, 0, 11, 10]
[12, 13, 18, 19, 17]
[0, 11, 2, 9, 12]
[9, 3, 13, 11,

In [37]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

prec  0.45066666666666666 recall  0.45066666666666666


In [38]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

F1_score: 0.45066666666666666


In [39]:
loaddata = np.load('../Data/latent_factor/0_0.npz')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/latent_factor/0_0.npz'

In [ ]:
User_latent_factor = loaddata['User']
YouTuber_latent_factor = loaddata['YouTuber']

In [ ]:
print(User_latent_factor.shape)
print(YouTuber_latent_factor.shape)

In [ ]:
User_latent_factor[139]

In [ ]:
YouTuber_latent_factor[139]

In [40]:
np.savez('../Data/latent_factor/YRM_up10/Final/1226.npz', User=U, YouTuber=Y)